In [1]:
!pip install qiskit

In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
import numpy as np
from math import pi, sqrt
import random

In [4]:
# PARAMETERS

# the basis gates
basis_g = ['id', 'rz', 'sx', 'x', 'cx']

#creating the gate statistics list
gate_stats = []

#number of qbits being controlled
ctrl_qbits = 0

# optimization level
optimization_lv = 3

In [5]:
# single qubit gates with no parameters
sgle_noparam = ["i","x","y","z","h","s","sdg","t","tdg","sx","sxdg"]

for gate in sgle_noparam:
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g, optimization_level = optimization_lv)
    gate_stats.append([gate, ctrl_qbits, transpile_circuit.size()])

In [6]:
# single qubit gates with 1 parameter
sgle_1_param = ["p","rx","ry","rz"]

for gate in sgle_1_param:
    param = random.uniform(-np.pi, np.pi)
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(param,0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g, optimization_level = optimization_lv)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

In [7]:
# single qubit gates with 2 parameters
single_2_param = ["u2"]

for gate in single_2_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-np.pi, np.pi)
    rand_angle2 = random.uniform(-np.pi, np.pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g, optimization_level = optimization_lv)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

/tmp/ipykernel_275/441393940.py:8: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, 0)


In [8]:
# single qubit gates with 3 parameters
single_3_param = ["u", "u3"]

for gate in single_3_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-pi, pi)
    rand_angle2 = random.uniform(-pi, pi)
    rand_angle3 = random.uniform(-pi, pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g, optimization_level = optimization_lv)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

/tmp/ipykernel_275/41438004.py:9: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)


In [9]:
gate_stats

[['i', 0, 0],
 ['x', 0, 1],
 ['y', 0, 2],
 ['z', 0, 1],
 ['h', 0, 3],
 ['s', 0, 1],
 ['sdg', 0, 1],
 ['t', 0, 1],
 ['tdg', 0, 1],
 ['sx', 0, 1],
 ['sxdg', 0, 3],
 ['p', 0, 1],
 ['rx', 0, 5],
 ['ry', 0, 4],
 ['rz', 0, 1],
 ['u2', 0, 3],
 ['u', 0, 5],
 ['u3', 0, 5]]

In [10]:
# creating a text file to store the gate stats(profiles)
import os
cd = os.getcwd()
path = cd + "/gate_stats.txt"
f = open(path, 'w')

In [11]:
# writing the profiles to text file
f = open("gate_stats" + "_op_lv_" + str(optimization_lv) + ".txt", "a")
for i in np.arange(len(gate_stats)):
    f.writelines([gate_stats[i][0], " ", str(gate_stats[i][1]), " ", str(gate_stats[i][2])])
    f.writelines(['\n'])
f.close()

In [13]:
# single qubit gates with no parameters
single_qubit_gates_no_params = ["x","y","z","h","s","sdg","t","tdg","sx","sxdg"]

# return gate with number of controls
def create_controlled_gate(circuit, controls, number_of_gate_qubits):
    # convert circuit to gate and add number of controls
    custom_circuit = circuit.to_gate().control(controls)

    # create new circuit with number of qubits for gate + for controls
    circuit_with_controls = QuantumCircuit(controls + number_of_gate_qubits)

    # append gate with controls to circuit with controls
    # second argument is a range for number of controls
    circuit_with_controls.append(custom_circuit, range(0, controls + number_of_gate_qubits))
    
    return circuit_with_controls


# create look up table for set of gates
# with different amounts of controls
def create_lookup_table(gates, max_controls):
    
    # iterate through each gate
    for gate in gates:
        
        # create circuit with all amount of controls
        for controls in range(0, max_controls + 1):
            
            # create single qubit circuit
            circuit = QuantumCircuit(1)
            
            # make circuit the actual gate
            # and pass 0 as a parameter
            getattr(circuit, gate)(0)
            
            if controls != 0:
                circuit_with_controls = create_controlled_gate(circuit, controls, 1)
                circuit = circuit_with_controls
                
            transpiled_circuit = transpile(circuit, basis_gates = basis_g)
            
            print(gate, transpiled_circuit.size(), controls)
            
create_lookup_table(single_qubit_gates_no_params, 5)

x 1 0
x 1 1
x 18 2
x 34 3
x 97 4
x 190 5
y 2 0
y 8 1
y 64 2
y 157 3
y 341 4
y 709 5
z 1 0
z 7 1
z 13 2
z 29 3
z 92 4
z 185 5
h 3 0
h 12 1
h 51 2
h 121 3
h 265 4
h 553 5
s 1 0
s 5 1
s 17 2
s 41 3
s 89 4
s 185 5
sdg 1 0
sdg 5 1
sdg 17 2
sdg 41 3
sdg 89 4
sdg 185 5
t 1 0
t 5 1
t 17 2
t 41 3
t 89 4
t 185 5
tdg 1 0
tdg 5 1
tdg 17 2
tdg 41 3
tdg 89 4
tdg 185 5
sx 1 0
sx 22 1
sx 89 2
sx 219 3
sx 483 4
sx 1011 5
sxdg 3 0
sxdg 22 1
sxdg 89 2
sxdg 219 3
sxdg 483 4
sxdg 1011 5
